## Import libraries

In [ ]:
# Classic stuff
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Keras imports
from keras.layers import *
from keras.applications import *
from keras import *
import tensorflow as tf
import tensorflow_hub as hub

### Check available devices

In [ ]:
# Check if gpu is available
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

## Global parameters

In [ ]:
# Dimensions of the image
IMG_HEIGHT = 224
IMG_WIDTH = 224

# Some parameters for the training and inference phase
STEPS_EPOCHS = 930
EPOCHS = 1
STEPS_PREDICT = 1861

## Creation of the dataset

In [ ]:
# Load and return an image
def load_image(img, training):
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32) / 127.5 - 1
    return tf.image.resize(img, (IMG_HEIGHT, IMG_WIDTH))

# Load the triplets
def load_triplets(triplet, training):
    ids = tf.strings.split(triplet)
    paths = ['food/' + ids[0] + '.jpg','food/' + ids[1] + '.jpg','food/' + ids[2] + '.jpg']
    anchor = load_image(tf.io.read_file(paths[0]), training)
    truthy = load_image(tf.io.read_file(paths[1]), training)
    falsy = load_image(tf.io.read_file(paths[2]), training)
    return tf.stack([anchor, truthy, falsy], axis=0),1
    
# Create the dataset using the input path
def create_dataset(dataset_filename):
    dataset = tf.data.TextLineDataset(dataset_filename)
    dataset = dataset.map(lambda triplet: load_triplets(triplet,True))
    return dataset

## Custom loss & accuracy function

In [ ]:
# Custom triplet loss
def triplet_loss(_,predictions): 
    anchor, correct, wrong = predictions[...,0],predictions[...,1],predictions[...,2]
    distance_correct = tf.reduce_sum(tf.square(anchor - correct),axis=1)
    distance_false = tf.reduce_sum(tf.square(anchor - wrong),axis=1)
    return tf.reduce_mean(tf.math.softplus(distance_correct - distance_false))
   
# Custom function to compute accuracy
def accuracy(_,predictions):
    anchor, correct, wrong = predictions[...,0],predictions[...,1],predictions[...,2]
    distance_correct = tf.reduce_sum(tf.square(anchor - correct),axis=1)
    distance_false = tf.reduce_sum(tf.square(anchor - wrong),axis=1)
    return tf.reduce_mean(tf.cast(tf.greater_equal(distance_false,distance_correct), tf.float32))

## Creation of the siamese nn

In [ ]:
# Creation of the model
def create_model():
    # Input layer
    input_layer = Input(shape=(3,IMG_HEIGHT, IMG_WIDTH, 3))
    
    # Split the layer into 3 parts
    anchor = input_layer[:,0,...]
    correct = input_layer[:,1,...]
    wrong = input_layer[:,2,...]
    
    # Use a pretrained encoder
    model_url = "https://tfhub.dev/google/bit/m-r50x1/1"
    encoder = hub.KerasLayer(model_url, trainable=False)
    
    # Create a custom classificator
    decoder = Sequential([
        Dropout(0.3),
        Dense(256),
        Activation('relu'),
        Dropout(0.3),
        Dense(128),
        Activation('sigmoid')       
    ])
    
    # Create the output layer
    output_layer = tf.stack([decoder(encoder(anchor)),decoder(encoder(correct)),decoder(encoder(wrong))],axis=-1)
    
    # Create & compile the model
    model = Model(inputs=input_layer,outputs=output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss = triplet_loss,
                   metrics=[accuracy])
    
    return model

# Create a model object
model = create_model()

## Predictor

In [ ]:
# Read the test data
def getTestData(dataset_filename):
    dataset = tf.data.TextLineDataset(dataset_filename)
    dataset = dataset.map(lambda triplet: load_triplets(triplet,False))
    dataset = dataset.batch(32).prefetch(10)
    return dataset

# Create a model for prediction
def create_inference_model(model):
    anchor, correct, wrong = model.output[...,0],model.output[...,1],model.output[...,2]
    distance_correct = tf.reduce_sum(tf.square(anchor - correct),1)
    distance_false = tf.reduce_sum(tf.square(anchor - wrong),1)
    predictions = tf.cast(tf.greater_equal(distance_false,distance_correct), tf.int8)
    return tf.keras.Model(inputs=model.inputs, outputs=predictions)

## Train the model

In [ ]:
# Load the train dataset
train_dataset = create_dataset('train_triplets.txt')
train_dataset = train_dataset.shuffle(1024, reshuffle_each_iteration=True).repeat().batch(64)

# Fit using this dataset
history = model.fit(
    train_dataset,
    steps_per_epoch= STEPS_EPOCHS,
    epochs=EPOCHS, # One epochs is enough
)

## Output model

In [ ]:
# Create the inference model
inference_model = create_inference_model(model)

# Load the data
input_data = getTestData('test_triplets.txt')

# Do the predictions
output = inference_model.predict(input_data,steps=STEPS_PREDICT)

# Output the predictions
np.savetxt('predictions.txt',output,fmt='%i')